In [4]:
import os
import json
from tqdm import tqdm
import numpy as np
from hydra import initialize, compose
from omegaconf import OmegaConf

from openelm.quality_metrics import utils
from openelm.mutation_model import PromptModel
from openelm.environments.p3.p3 import P3ProbSol_Chat_PP

In [8]:
path_high = '/home/laetitia/work/code-eval/data/puzzles_high_fitness_archivetrain.json'
path_low = '/home/laetitia/work/code-eval/data/puzzles_low_fitness_archivetrain.json'
p_high = json.load(open(path_high, 'r'))
p_low = json.load(open(path_low, 'r'))
all_puzzles = p_high + p_low

In [11]:
lehmer = [pz for pz in all_puzzles if 'name' in pz and 'Lehmer' in pz['name']][0]
print(lehmer['program_str'])

def f(n: int):
    """Find n  such that 2^n mod n = 3"""
    return pow(2, n, n) == 3
def g():
    return 4700063497
assert f(g()) == True


In [6]:
with initialize(version_base="1.2"):
    cfg = compose(
        config_name="elmconfig",
        overrides=["env=P3ProbSolChatEnv_PP_ELM_NLP"]
    )
    # print(cfg)
config = OmegaConf.to_object(cfg)

cfg_generation: dict = {
            "temperature": 0.,
            "model": config.model.model_path,
        }
print(config.model.model_path)

gpt-3.5-turbo-0125


In [7]:
mutation_model = PromptModel(config.model)
env = P3ProbSol_Chat_PP(config=config.env,
        mutation_model=mutation_model)

 



none 





load embedding model:
ChatGPT
loading preprocessed trainset


100%|██████████| 133/133 [00:00<00:00, 682793.67it/s]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


bsize 2
LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32256, 2048)
    (layers): ModuleList(
      (0-23): 24 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (v_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): LlamaLinearScalingRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2048, out_features=5504, bias=False)
          (up_proj): Linear(in_features=2048, out_features=5504, bias=False)
          (down_proj): Linear(in_features=5504, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm(

In [12]:
puzzle, solution = utils.get_puzzle_sol(lehmer)

In [13]:
final_losses = env._get_losses(puzzle, solution)

In [17]:
final_losses.tolist().index(min(final_losses))

36

In [18]:
env.archive_puzzle_strs[36]

'def f(s: str, target="foobarbazwow", length=6):\n    """Find a substring of the given length centered within the target string."""\n    return target[(len(target) - length) // 2:(len(target) + length) // 2] == s'

In [22]:
l = [(i, p) for i, p in enumerate(env.archive_puzzle_strs) if 'pow(2, n, n)' in p]

In [23]:
l

[(3,
  'def f(n: int):\n    """Find n  such that 2^n mod n = 3"""\n    return pow(2, n, n) == 3')]

In [24]:
final_losses[3]

tensor(2.7380)

In [25]:
max(final_losses)

tensor(3.0754)

In [26]:
p0 = all_puzzles[0]

In [27]:
final_losses = env._get_losses(*utils.get_puzzle_sol(p0))


In [28]:
p0['name']

'PenultimateRevString:2'